In [13]:
import tensorflow as tf
from model import LabelEncoder, Anchor
from utils import data_preprocess, data_generator
import numpy as np 
import tensorflow_datasets as tfds
import model.Model as Model
import model.Loss as Loss
import os

data_dir = r'F:\dataset\kitti\data_object_image_2\training'
model_dir = r'retinanet\\'
label_encoder = LabelEncoder.LabelEncoder()

num_classes = 8
batch_size = 8


In [2]:
# get data list
img_list = tf.io.gfile.glob(f'{data_dir}\image_2\*.png')
label_list = tf.io.gfile.glob(f'{data_dir}\label_2\*.txt')
img_list.sort()
label_list.sort()

In [3]:
# img, labels, classes = data_preprocess.prepare_data(img_list[0],label_list[0])
# ax = data_preprocess.visualize_detections(img, labels)

In [4]:


learning_rate = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values = learning_rate
)

In [5]:
dataset = data_generator.get_dataset(img_list, label_list)
dataset = dataset.shuffle(32)

dataset = dataset.padded_batch(
    batch_size = batch_size, padding_values = (0.0, 1e-8, -1.0), drop_remainder=True
)
dataset = dataset.map(label_encoder.encode_batch)
dataset = dataset.apply(tf.data.experimental.ignore_errors())
dataset = dataset.prefetch(tf.data.AUTOTUNE)
train_dataset = dataset.take(100)
val_dataset = dataset.skip(100).take(100)

In [6]:
# sample = next(iter(train_dataset))
# print(sample[0][0].shape)
# ax = data_preprocess.visualize_detections(sample[0][0], sample[0][1])

In [7]:
resnet50_backbone = Model.get_backbone()
loss_fn = Loss.RetinaNetLoss(num_classes)
model = Model.RetinaNet(num_classes, backbone = resnet50_backbone)

optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss=loss_fn, optimizer= optimizer)

In [8]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

In [9]:
model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=10,
    callbacks = callbacks_list,
)

Epoch 1/10
    100/Unknown - 108s 889ms/step - loss: 0.7139
Epoch 1: saving model to retinanet\weights_epoch_1
100/100 [==============================] - 198s 2s/step - loss: 0.7139 - val_loss: 0.4141
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.7036
Epoch 2: saving model to retinanet\weights_epoch_2
100/100 [==============================] - 130s 1s/step - loss: 0.7036 - val_loss: 0.4199
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.6726
Epoch 3: saving model to retinanet\weights_epoch_3
100/100 [==============================] - 128s 1s/step - loss: 0.6726 - val_loss: 0.4101
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 0.6647
Epoch 4: saving model to retinanet\weights_epoch_4
100/100 [==============================] - 124s 1s/step - loss: 0.6647 - val_loss: 0.3956
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 0.6876
Epoch 5: saving model to retinanet\weights_epoch_5
100/100 [========

In [15]:
from model.Model import DecodePredictions
weights_dir = 'retinanet'

latest_checkpoint = tf.train.latest_checkpoint(weights_dir)
model.load_weights(latest_checkpoint)

image = tf.keras.Input(shape=[None, None, 3], name='image')
predictions = model(image, training=False)
detections = DecodePredictions(confidence_threshold = 0.5, num_classes = num_classes)(image, predictions)
inference_model = tf.keras.Model(inputs= image, outputs=detections)


In [ ]:

sample = next(iter(val_dataset))
inference_model.predict(sample[0])